<a href="https://colab.research.google.com/github/purang2/prompting-nlp/blob/main/KoGPT_TextChat2Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **[ Produced by ]**

[Eunchan Lee of KNU-BrainAI](https://github.com/purang2)


**[ 목차 ]**


---



### Requirements 설치 

In [ ]:
import gc
import torch 

  
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!pip install transformers 
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 4.0 MB/s 
     |████████████████████████████████| 120 kB 91.8 MB/s 
     |████████████████████████████████| 6.6 MB 58.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 143 kB 4.0 MB/s 


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

## Model Loading

Huggingface/transformers 패키지를 통해 토크나이저와 모델을 load 

Model size: 11.5 GB (큰 것 같다 -> 16GB 권장, 고용량의 RAM 필요)
- KoGPT6B-ryan1.5b는 32GB GPU RAM을 최소로 권장 
- KoGPT6B-ryan1.5b-float16는 16GB GPU RAM을 권장


-> it takes about 5 min

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16', # or float32 version: revision=KoGPT6B-ryan1.5b
    bos_token = '[BOS]', eos_token='[EOS]', unk_token = '[UNK]', pad_token='[PAD]', mask_token = '[MASK]'
)

model = AutoModelForCausalLM.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',# or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

_  = model.eval()

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.3G [00:00<?, ?B/s]

In [ ]:
answer_set = [] #answer set 


#prompt 

'''
#1: 아래 지문을 읽고 물음에 답을 유추하시오. 
#2: 질의응답 태스크 수행: 
#3: {맥락} \n {질문}에 대해 답이 뭔지 아니? 

'''
import gc
import torch 

  
gc.collect()
torch.cuda.empty_cache()

prompt_with_query = ''



#For Naive-Conversation dataset 
txt = '''
입력: 야 뭐해 
출력: 나 밥먹고 있지 너는?

입력: 나는 노래 듣고 카페에 있어 밥 뭐 먹냐?
출력: 피자 어제 남은거 먹었어 

입력: 맛있었음?
출력: 그럭저럭임 ㅋㅋ 수업있냐?

입력: ㅇㅇ 학교가는중 학교 언제옴?
출력: 이제 일어남 ㅋㅋㅋ 오늘 수업 있냐

입력: ㅇㅇ 10시 반 수업임 언제 출발하냐? 같이 갈래?
'''

#[ { "role": "user", "text": "담배를 피울 수 있는 비싼 가격대의 에어비엔비를 찾고 있습니다. 지역은 상관없습니다.", "state": [ "숙소-가격대-비싼", "숙소-종류-에어비엔비", "숙소-지역-dontcare", "숙소-흡연 가능-yes" ] }, { "role": "sys", "text": "혹시 다른 종류의 숙소는 생각이 없으신가요? 현재 그 조건으로는 찾을 수 없습니다.", "state": [] }, { "role": "user", "text": "그럼 호텔로는 있을까요?", "state": [ "숙소-가격대-비싼", "숙소-종류-호텔", "숙소-지역-dontcare", "숙소-흡연 가능-yes" ] }, { "role": "sys", "text": "네 명동역 근처에 있는 평점 4.9점에 프린스 호텔 서울을 찾았습니다.", "state": [] }, { "role": "user", "text": "그럼 그 곳으로 월요일부터 9일간 한명을 예약해주세요.", "state": [ "숙소-가격대-비싼", "숙소-종류-호텔", "숙소-지역-dontcare", "숙소-흡연 가능-yes", "숙소-예약 요일-월요일", "숙소-예약 명수-1", "숙소-예약 기간-9", "숙소-이름-프린스 호텔 서울" ] }, { "role": "sys", "text": "네 예약되었습니다.", "state": [] }, { "role": "user", "text": "그럼 예약번호랑 주소, 전화번호까지 알려주세요.", "state": [ "숙소-가격대-비싼", "숙소-종류-호텔", "숙소-지역-dontcare", "숙소-흡연 가능-yes", "숙소-예약 요일-월요일", "숙소-예약 명수-1", "숙소-예약 기간-9", "숙소-이름-프린스 호텔 서울" ] }, { "role": "sys", "text": "예약번호는 'QVJO3'이고 주소는 서울 중구 60927이고 전화번호는 626919822입니다.", "state": [] }, { "role": "user", "text": "아 그리고 서울 중앙에 비싼 가격대의 한식당을 찾고 있습니다.", "state": [ "숙소-가격대-비싼", "숙소-종류-호텔", "숙소-지역-dontcare", "숙소-흡연 가능-yes", "숙소-예약 요일-월요일", "숙소-예약 명수-1", "숙소-예약 기간-9", "숙소-이름-프린스 호텔 서울", "식당-가격대-비싼", "식당-지역-서울 중앙", "식당-종류-한식당" ] }, { "role": "sys", "text": "네 찾았습니다. 제가 추천드리고 싶은 곳은 평점 4.3점에 명동역에서 도보 7분거리에 있는 외계인의맛집입니다.", "state": [] }, { "role": "user", "text": "오 거기 그 식당에 월요일에 14시30분에 한명예약해주세요.", "state": [ "숙소-가격대-비싼", "숙소-종류-호텔", "숙소-지역-dontcare", "숙소-흡연 가능-yes", "숙소-예약 요일-월요일", "숙소-예약 명수-1", "숙소-예약 기간-9", "숙소-이름-프린스 호텔 서울", "식당-가격대-비싼", "식당-지역-서울 중앙", "식당-종류-한식당", "식당-예약 요일-월요일", "식당-예약 시간-14:30", "식당-예약 명수-1", "식당-이름-외계인의맛집" ] }, { "role": "sys", "text": "네 예약번호 'QJWF6'으로 예약되었습니다.", "state": [] }, { "role": "user", "text": "아 근데 거기에 흡연석이 있나요?", "state": [ "숙소-가격대-비싼", "숙소-종류-호텔", "숙소-지역-dontcare", "숙소-흡연 가능-yes", "숙소-예약 요일-월요일", "숙소-예약 명수-1", "숙소-예약 기간-9", "숙소-이름-프린스 호텔 서울", "식당-가격대-비싼", "식당-지역-서울 중앙", "식당-종류-한식당", "식당-예약 요일-월요일", "식당-예약 시간-14:30", "식당-예약 명수-1", "식당-이름-외계인의맛집" ] }, { "role": "sys", "text": "외계인의맛집에는 흡연이 가능합니다.", "state": [] }, { "role": "user", "text": "아. 감사합니다.", "state": [ "숙소-가격대-비싼", "숙소-종류-호텔", "숙소-지역-dontcare", "숙소-흡연 가능-yes", "숙소-예약 요일-월요일", "숙소-예약 명수-1", "숙소-예약 기간-9", "숙소-이름-프린스 호텔 서울", "식당-가격대-비싼", "식당-지역-서울 중앙", "식당-종류-한식당", "식당-예약 요일-월요일", "식당-예약 시간-14:30", "식당-예약 명수-1", "식당-이름-외계인의맛집" ] }, { "role": "sys", "text": "감사합니다.", "state": [] } ]



#For Wizard-of-Seoul (WoS, = KLUE-DST)
txt='''

{ "담배를 피울 수 있는 비싼 가격대의 에어비엔비를 찾고 있습니다. 지역은 상관없습니다.", [ "숙소-가격대-비싼", "숙소-종류-에어비엔비", "숙소-지역-dontcare", "숙소-흡연 가능-yes" ] } 
{ "혹시 다른 종류의 숙소는 생각이 없으신가요? 현재 그 조건으로는 찾을 수 없습니다.", [] }
{ "그럼 호텔로는 있을까요?", [ "숙소-가격대-비싼", "숙소-종류-호텔", "숙소-지역-dontcare", "숙소-흡연 가능-yes" ] }
{ "네 명동역 근처에 있는 평점 4.9점에 프린스 호텔 서울을 찾았습니다.", [] }
{ "그럼 그 곳으로 월요일부터 9일간 한명을 예약해주세요.", [ "숙소-가격대-비싼", "숙소-종류-호텔", "숙소-지역-dontcare", "숙소-흡연 가능-yes", "숙소-예약 요일-월요일", "숙소-예약 명수-1", "숙소-예약 기간-9", "숙소-이름-프린스 호텔 서울" ] }
{ "네 예약되었습니다."[] } 
{ "그럼 예약번호랑 주소, 전화번호까지 알려주세요.", [ "숙소-가격대-비싼", "숙소-종류-호텔", "숙소-지역-dontcare", "숙소-흡연 가능-yes", "숙소-예약 요일-월요일", "숙소-예약 명수-1", "숙소-예약 기간-9", "숙소-이름-프린스 호텔 서울" ] }
{ "예약번호는 'QVJO3'이고 주소는 서울 중구 60927이고 전화번호는 626919822입니다.", [] }
{ "아 그리고 서울 중앙에 비싼 가격대의 한식당을 찾고 있습니다.",  
'''

txt ='''
입력: 길에 고양이가 두 마리 있다.
출력: 저 고양이 너무 귀엽다!

입력: 나무에서 낙엽이 떨어졌다.
출력: 낙엽이 떨어지네? 

입력: 한 사람이 코딩을 하고 있다. 
출력: 저 사람 코딩하는 모습이 멋지다 ㅋㅋ

입력: 푸른 하늘에 푸르른 나무들이 있다.
출력: 사진 잘 찍었네 하늘이 예쁘다!

입력: 한 사람이 카페에서 주문을 하고 있다.
'''

# For SKT Text-Style-Transfer
#PARAMS
# gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=512)
 
txt  = '''
입력: "안녕하세요, 일러스트 그림 그려주세요. 후드티를 입고 고양이 귀를 한 분홍머리 여자, 나이는 16살 정도, 일본 애니메이션 스타일로 부탁드려요.
출력: 일러스트레이션, 후드티, 고양이 귀, 분홍머리, 여자, 16살, 일본 애니메이션 스타일

입력: "웹툰 표지로 일러스트를 쓰고싶은데 그림 그려줘. 웹툰은 좀비 던전과 보물에 얽힌 비밀들에 관한 내용이고, 내가 필요한 일러스트는 좀비 던전을 지나는 칼을 든 금발의 5살 여자 아이야.
출력: 일러스트레이션, 웹툰, 좀비 던전, 칼을 든, 금발, 5살, 여자 아이

입력: "추석이기도 하니. 보름달을 배경으로 한 한복입고 세배하는 토끼를 그려줄 수 있을까?"
출력: 일러스트레이션, 고화질, 보름달 배경, 한복, 세배하는, 토끼

입력: "그림을 그려주세요. 음 기타를 든 잘생기고 젊은 가수로 부탁하고, 배경은 대학교 축제로 해주세요!"
출력:


'''


prompt_with_query += txt

with torch.no_grad():
  tokens = tokenizer.encode(prompt_with_query, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=512)
  generated = tokenizer.batch_decode(gen_tokens)[0]

#print(generated[len(prompt):])
answer_set.append(generated[generated.rfind("출력:"):])
print(generated)


입력: "안녕하세요, 일러스트 그림 그려주세요. 후드티를 입고 고양이 귀를 한 분홍머리 여자, 나이는 16살 정도, 일본 애니메이션 스타일로 부탁드려요.
출력: 일러스트레이션, 후드티, 고양이 귀, 분홍머리, 여자, 16살, 일본 애니메이션 스타일

입력: "웹툰 표지로 일러스트를 쓰고싶은데 그림 그려줘. 웹툰은 좀비 던전과 보물에 얽힌 비밀들에 관한 내용이고, 내가 필요한 일러스트는 좀비 던전을 지나는 칼을 든 금발의 5살 여자 아이야.
출력: 일러스트레이션, 웹툰, 좀비 던전, 칼을 든, 금발, 5살, 여자 아이

입력: "추석이기도 하니. 보름달을 배경으로 한 한복입고 세배하는 토끼를 그려줄 수 있을까?"
출력: 일러스트레이션, 고화질, 보름달 배경, 한복, 세배하는, 토끼

입력: "그림을 그려주세요. 음 기타를 든 잘생기고 젊은 가수로 부탁하고, 배경은 대학교 축제로 해주세요!"
출력:









일러스트레이션, 배경, 가수, 대학교 축제, 배경, 기타











* * *
일러스트레이션으로 디자인한 그림을 출력해드립니다.
출력물은 A4사이즈로 제작됩니다.
그림은 직접 그려서 보내주셔도 됩니다.

* * *
완성된 콘텐츠를 페이스북, 인스타그램, 이메일로 전달해드립니다.

• * *
완성된 콘텐츠를 PDF파일로 보내드립니다.

* * *
완성된 콘텐츠를 PDF파일로 받아보실 수 있습니다.




• * * *
완성된 콘텐츠를 카드뉴스형 콘텐츠로 보내드립니다.

• * * *
완성된 콘텐츠를 카드뉴스형 콘텐츠로 받아보실 수 있습니다.




• * * *
완성된 콘텐츠를 PPT파일로 보내드립니다.[EOS]
